In [1]:
using GamsStructure

using JuMP.Containers

In [2]:
L = [1,2,3,4]

[[c] for c in L]

4-element Vector{Vector{Int64}}:
 [1]
 [2]
 [3]
 [4]

In [ ]:
struct GP{N}
    universe
    sets::Tuple{Vararg{Symbol}}
    value::Array{Float64,N}
    description::String
    GP(GU,sets::Tuple{Vararg{Symbol}},description::String) = new{length(sets)}(GU,sets,zeros(Float64,Tuple(length(GU[e]) for e∈sets)),description)
    GP(GU,sets::Tuple{Vararg{Symbol}};description = "") = new{length(sets)}(GU,sets,zeros(Float64,Tuple(length(GU[e]) for e∈sets)),description)
end


function Base.show(io::IO,P::GP)
    print("Description: $(P.description)\nDomain: $(P.sets)\n\n")
    show(P.value)
end

In [ ]:
function convert_idx(P::GP,i::Int,idx::Union{Symbol,Colon})
    GU = P.universe
    set = P.sets[i]
    return collect(1:length(GU[set]))
end

function convert_idx(P::GP,i::Int,idx::Vector{Symbol})
    GU = P.universe
    set = indexin(idx,[e for e in GU[P.sets[i]]])#findall(x-> x∈idx, GU[P.sets[i]])
    if length(set) == 1
        return set[1]
    end
    return set
end

function convert_idx(P::GP,i::Int,idx::Vector{Bool})
    #GU = P.universe
    #set = P.sets[i]
    return idx
end

function convert_idx(P::GP,i::Int,idx)
    return idx
end

function Base.getindex(P::GP,idx...)
    idx = map(x->convert_idx(P,x[1],x[2]),enumerate(idx))
    return P.value[idx...]
end


function Base.iterate(iter::GP)
    next = iterate(iter.value)
    return next === nothing ? nothing : (next[1], next[2])
end

function Base.iterate(iter::GP, state)
    next = iterate(iter.value, state)
    return next === nothing ? nothing : (next[1], next[2])
end


function Base.setindex!(X::GP,value,idx...)
    new_index = map(x->convert_idx(X,x[1],x[2]),enumerate(idx))
    X.value[new_index...] = value
end

function Base.length(X::GP)
    return length(X.value)
end

In [ ]:
GU = GamsUniverse()

@GamsSet(GU,:i,"Test",begin
    a, ""
    b, ""
end)

X = GP(GU,(:i,:i),"This is a test")

X[[:a],[:a]] = 1
X[[:a],[:b]] = 2
X[[:b],[:a]] = 3
X[[:b],[:b]] = 4

X

In [ ]:
c = findall(X.>2)
X[c]

In [ ]:
X[[:a],[:a]]

In [ ]:
X[findall(X .>2)]=[10,11]

In [ ]:
X

In [ ]:
findall(x->x>0,Y)

In [ ]:
X[:i,:i] = X[:i,:i].+X[:i,:i]

In [ ]:
for i∈GU[:i]
    println("$i -> $(sum(X[:i,[i]]))") #Careful here
end

In [ ]:
X[:i,:i] + X[:i,:i]

In [ ]:
for i∈GU[:i],j∈GU[:i]
    println(X[[i],[j]])
end

In [ ]:
string(X.value)

In [ ]:
X.value